In [1]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo

In [2]:
dry_bean_dataset = fetch_ucirepo(id=602) 
  
X = dry_bean_dataset.data.features 
y = dry_bean_dataset.data.targets 

In [3]:
X_arr = np.array(X.iloc[:,:])

X_arr

array([[2.83950000e+04, 6.10291000e+02, 2.08178117e+02, ...,
        3.14728900e-03, 8.34222388e-01, 9.98723889e-01],
       [2.87340000e+04, 6.38018000e+02, 2.00524796e+02, ...,
        3.56362400e-03, 9.09850506e-01, 9.98430331e-01],
       [2.93800000e+04, 6.24110000e+02, 2.12826130e+02, ...,
        3.04773300e-03, 8.25870617e-01, 9.99066137e-01],
       ...,
       [4.21390000e+04, 7.59321000e+02, 2.81539928e+02, ...,
        1.88827100e-03, 6.76884164e-01, 9.96767264e-01],
       [4.21470000e+04, 7.63779000e+02, 2.83382636e+02, ...,
        1.85202500e-03, 6.68236684e-01, 9.95222420e-01],
       [4.21590000e+04, 7.72237000e+02, 2.95142741e+02, ...,
        1.63981200e-03, 6.16220592e-01, 9.98179623e-01]])

In [4]:
classes = list(y['Class'].value_counts().index)

for c in classes:
    y.loc[:,c] = y['Class'].apply(lambda x: 1 if x == c else 0)

y_arr = np.array(y.iloc[:,1:])

y_arr

C:\Users\advai\AppData\Local\Temp\ipykernel_33052\840677549.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.loc[:,c] = y['Class'].apply(lambda x: 1 if x == c else 0)
C:\Users\advai\AppData\Local\Temp\ipykernel_33052\840677549.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.loc[:,c] = y['Class'].apply(lambda x: 1 if x == c else 0)
C:\Users\advai\AppData\Local\Temp\ipykernel_33052\840677549.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_arr, y_arr, test_size=0.3, random_state=42)

In [7]:
y_arr[0]

array([0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [39]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def relu(x):
    return np.maximum(x,0)

def tanh(x):
    return np.tanh(x)

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [43]:
learning_rate = 0.1
epochs = 750

input_hidden1_weights = np.random.uniform(-1,1,(16,48))
bias_hidden1_weights = np.random.uniform(-1,1,(1,48))
hidden1_hidden2_weights = np.random.uniform(-1,1,(48,24))
bias_hidden2_weights = np.random.uniform(-1,1,(1,24))
hidden2_hidden3_weights = np.random.uniform(-1,1,(24,12))
bias_hidden3_weights = np.random.uniform(-1,1,(1,12))
hidden3_output_weights = np.random.uniform(-1,1,(12,7))

for epoch in range(epochs):
    x1 = np.matmul(X_train,input_hidden1_weights) + bias_hidden1_weights
    x1 = tanh(x1)
    
    x2 = np.matmul(x1, hidden1_hidden2_weights) + bias_hidden2_weights
    x2 = relu(x2)

    x3 = np.matmul(x2, hidden2_hidden3_weights) + bias_hidden3_weights
    x3 = sigmoid(x3)
    
    y_pred = np.matmul(x3, hidden3_output_weights)
    y_pred = softmax(y_pred)
    
    error = y_train - y_pred
    
    delK = error * (y_pred * (1-y_pred))
    delJ = np.matmul(delK, hidden3_output_weights.T) * (x3 * (1-x3))
    delI = np.matmul(delJ, hidden2_hidden3_weights.T) * (x2 > 0)
    delH = np.matmul(delI, hidden1_hidden2_weights.T) * (1 - x1)*(1 + x1)
    
    
    hidden3_output_weights += learning_rate * (x3.T @ delK)
    hidden2_hidden3_weights += learning_rate * np.matmul(x2.T, delJ)
    hidden1_hidden2_weights += learning_rate * np.matmul(x1.T, delI)
    input_hidden1_weights += learning_rate * np.matmul(X_train.T, delH)
    bias_hidden1_weights += learning_rate * np.sum(delH, axis=0)
    bias_hidden2_weights += learning_rate * np.sum(delI, axis=0)
    bias_hidden3_weights += learning_rate * np.sum(delJ, axis=0)
    
    loss = np.mean(np.square(error))
    print(f"Epoch: {epoch + 1}, Loss: {loss}",end="\r")

In [36]:
# Testing
x1 = sigmoid(np.matmul(X_test, input_hidden1_weights))
x2 = sigmoid(np.matmul(x1, hidden1_hidden2_weights))
x3 = sigmoid(np.matmul(x2, hidden2_hidden3_weights))
y_pred = sigmoid(np.matmul(x3, hidden3_output_weights))

C:\Users\advai\AppData\Local\Temp\ipykernel_33052\1795901080.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


In [37]:
idx = y_pred.argmax(axis=1)
out = (idx[:,None] == np.arange(y_pred.shape[1])).astype(float)

out

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [38]:
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(y_true=y_test, y_pred=out))

0.25538687561214496
